In [1]:
!git clone https://github.com/sam12sam12sam/boost_test.git

Cloning into 'boost_test'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 100 (delta 21), reused 15 (delta 4), pack-reused 31 (from 2)
Receiving objects: 100% (100/100), 42.49 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Updating files: 100% (19/19), done.


In [2]:
!ls

boost_test  sample_data


In [3]:
%cd boost_test

/content/boost_test


In [4]:
!ls

Etl_Pipeline  ref


In [5]:
%cd Etl_Pipeline

/content/boost_test/Etl_Pipeline


In [6]:
!ls

data  etl  notebooks  README.md  requirements.txt  sql


In [7]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.4 MB/s eta 0:00:00


In [8]:
from etl.db import get_engine

engine = get_engine()
engine.connect()


In [9]:
!ls etl

config.py  ingest	load	   main.py	transform
db.py	   __init__.py	logger.py  __pycache__


In [10]:
from etl.db import get_engine
from etl.ingest.csv_ingest import ingest_csv
from etl.ingest.json_ingest import ingest_json
from etl.transform.csv_cleaner import clean_csv
from etl.transform.json_cleaner import clean_json
from etl.load.mysql_loader import load_df

engine = get_engine()

# CSV
raw_csv = ingest_csv("data/test.csv")
load_df(raw_csv, "raw_test", engine)

clean_csv_df = clean_csv(raw_csv)
load_df(clean_csv_df, "test", engine)

# JSON
import json
import pandas as pd
raw_json = ingest_json("data/test.json")
load_df(raw_json, "raw_users", engine)

users, phones, jobs = clean_json(raw_json)

users['created_at'] = pd.to_datetime(users['created_at'], unit="s", errors="coerce")
users['updated_at'] = pd.to_datetime(users['updated_at'], unit="s", errors="coerce")
users['dob'] = pd.to_datetime(users['dob'], errors="coerce").dt.date
load_df(users, "users", engine)

load_df(phones, "telephone_numbers", engine)

jobs['logged_at'] = pd.to_datetime(jobs['logged_at'], unit="s", errors="coerce")
jobs['start'] = pd.to_datetime(jobs['start'], errors="coerce").dt.date
jobs['end'] = pd.to_datetime(jobs['end'], errors="coerce").dt.date
jobs['is_fulltime'] = jobs['is_fulltime'].astype(str).str.lower().isin(["true", "1"])
load_df(jobs, "jobs_history", engine)